# Combined Optimizations
Now that we know how to make decisions on choosing minimal numbers of suppliers and earning maximal net profit, it is time to combine these 2 optimization problems together and solve the problem in a more realistic situation.

Now the grocer needs to maintain the inventory from smallest number of suppliers while chasing the largest profit. This can be modelled by combining the "Set Cover" problem from supplier optimization and "Knapsack" problem from profit optimization. As always, notations and assumptions are first given as follows:

  1. there are $N$ different kinds of items $U=\{ U_0, U_1,\cdots,U_{N-1} \}$
  2. each of the item has its own selling price $V=\{ V_0, V_1, \cdots, V_{N-1}\}$
  3. there are $M$ different suppliers $S=\{ S_0, S_1, \cdots, S_{M-1}\}$
  4. costs are different from suppliers $W_j=\{ W_{j0}, W_{j1}, \cdots, W_{j(N-1)}\}$ with $j=0,\cdots,M-1$
  5. the quantity of each item has upper bounds $B=\{ B_0, B_1, \cdots, B_{N-1}\}$
  6. there is a budget limit $C$ for buying the items
  7. all items will be sold out

This combination of "Set Cover" problem and "Knapsack" problem can be handled by D-Wave CQM solver.

## Import Labraries

In [1]:
from dimod import CQM, Binary, Integer, quicksum
from dwave.system import LeapHybridCQMSampler
import numpy as np
import pandas as pd

## Problem Set Up
Randomly generating data as shown in the assumption.

In [2]:
# ---------- Problem set up -------------
np.random.seed(10) # fixed randomness

print('------------- Problem set up -------------')
# inventory universe
U = list(set(np.random.randint(16, size=(10))))
# Average cost of items
W_avg = {U[i]: int(np.random.randint(9)+1) for i in range(len(U))}
# Price of items
V = [int(1.2*W_avg[U[i]]) for i in range(len(U))]
# Upper bound on number of items
upper_bound = list(np.random.randint(2, 8, size=(len(U))))
# Budget
Wbudget = 100
# Print set up
print('------------- Inventory -------------')
print('Budget:', Wbudget)
print('Number of elements in the universe: {:d}'.format(len(U)))
print('The inventory universe is', U)
print('The price of each item is', V)
print('Bound on item quantity is', upper_bound)
print('------------------------------------------')



------------- Problem set up -------------
------------- Inventory -------------
Budget: 100
Number of elements in the universe: 8
The inventory universe is [0, 1, 4, 9, 11, 12, 13, 15]
The price of each item is [1, 2, 10, 1, 10, 8, 6, 4]
Bound on item quantity is [2, 6, 5, 2, 5, 4, 3, 2]
------------------------------------------


In [3]:
# suppliers 
S = [set(U[i] for i in np.random.randint(len(U), size=(6))) for j in range(5)]
# Costs from each supplier
W = list(S)
for i in range(len(S)):
    W[i] = list(S[i])
    for j in range(len(W[i])):
        W[i][j] = int(np.random.randint(75,125,1)*W_avg[list(S[i])[j]]/100) + 1
# Print set up
print('------------- Suppliers -------------')
print('There are {:d} suppliers:'.format(len(S)))
print('----------------------------')
for i in range(len(S)):
    print('Supplier{:d}'.format(i), S[i])
    print('Costs: ', W[i])
    print('----------------------------')
print('------------------------------------------')

------------- Suppliers -------------
There are 5 suppliers:
----------------------------
Supplier0 {1, 9, 11, 12, 13}
Costs:  [2, 2, 8, 7, 5]
----------------------------
Supplier1 {9, 12, 13, 1}
Costs:  [1, 7, 5, 2]
----------------------------
Supplier2 {1, 11, 13}
Costs:  [2, 8, 5]
----------------------------
Supplier3 {1, 4, 9, 11, 15}
Costs:  [3, 8, 1, 10, 5]
----------------------------
Supplier4 {0, 12, 13, 15}
Costs:  [1, 8, 5, 5]
----------------------------
------------------------------------------


## Building CQM
Both binary variables $y_j$ and discrete variables $x_{ji}$ are needed to build the model.
Binary variables $y_j$ indicate the decision for choosing supplier $S_j$, i.e.,

$$
y_j = 
\left\{
    \begin{array}{cl}
        1, & S_j \mathrm{\ \ is\ \ chosen}\\
        0, & \mathrm{otherwise}
    \end{array}
\right.
$$

Discrete variables $x_{ji}$ denote the quantity of restoring item $U_i$ from supplier $S_j$. A loose upper bound for $x_{ji}$ is obtained from the item quantity upper bound:

$$
0\leq x_{ji}\leq B_i,\ \ \ \ i=0,\cdots,N-1.
$$

In [4]:
# ---------- Build QCM ------------
cqm = CQM()

# Create discrete variables
# Quantity of each item from each supplier
x = [[Integer((j,i), upper_bound=upper_bound[i]) for i in range(len(U))] for j in range(len(S))]
# Create binary variables
# Indicator for choosing each supplier
y = [[Binary((j,len(U)+1))] for j in range(len(S))]


The main target is no longer unique in this situation: chasing maximal net profit while using minimal number of suppliers. What's worse, these 2 goals may not be consistant with each other: Reducing the number of suppliers may increase the costs for restoring inventory. There are ways to handle this problem, and a normal way is to assign "weights" on these 2 targets, which gives the following objective function:

$$
\min\ \  A\cdot\sum_{i,j=0}^{N-1,M-1} (W_{ji}-V_i)\cdot x_{ji}\cdot y_j \ +\  B\cdot\sum_{j=0}^{M-1} y_j
$$

where $A,B$ are Lagrange Multipliers for weighting the "importance" of the targets.

In [5]:
# -------------- Objective Function ------------------
# maximize total profit
obj1 = -quicksum(V[i]*x[j][i]*y[j][0] for i in range(len(U)) for j in range(len(S)) )
# minimize total cost
obj2 = quicksum(W[j][i]*x[j][ U.index(list(S[j])[i]) ]*y[j][0] for j in range(len(S)) for i in range(len(W[j])) )
# minimize number of suppliers
obj3 = quicksum(y[j][0] for j in range(len(S)))

# set Lagrange Multipliers
A = 1
B = 1

# Add obj to CQM
cqm.set_objective(A*(obj1+obj2)+B*obj3)

There are several constraints for this problem:

    1. chosen suppliers should cover all items from inventory

In [6]:
# -------------- Constraints -----------------
# suppliers should cover all items
for i in range(len(U)):
    cqm.add_constraint( quicksum( int(U[i] in S[j])*x[j][i]*y[j][0] for j in range(len(S))) >= 1,
                        label = 'cover item {:d}'.format(i) )

    2. costs should not exceed the total budget limit

In [7]:
# no exceeding total budget
cqm.add_constraint( quicksum(W[j][i]*x[j][ U.index(list(S[j])[i]) ]*y[j][0] for j in range(len(S)) for i in range(len(W[j])) ) <= Wbudget,
                    label = 'budget' )

'budget'

    3. each item has a total upper bound from the grocer

In [8]:
# no exceeding upper bound
for i in range(len(U)):
    cqm.add_constraint( quicksum(x[j][i] for j in range(len(S))) <= upper_bound[i], 
                        label = 'bound item {:d}'.format(i) )

    4. grocer cannot buy items that are not supplied from the supplier

In [9]:
# cannot buy without supply
for i in range(len(U)):
    for j in range(len(S)):
        if U[i] not in S[j]:
            cqm.add_constraint( x[j][i] == 0, 
                                label = 'supplier{:d} sells no item{:d}'.format(j, U[i]) )


## Submit to CQM Solver

In [10]:
# -------------- Submit to CQM sampler ---------------
cqm_sampler = LeapHybridCQMSampler()
sampleset = cqm_sampler.sample_cqm(cqm, label = 'Combine Demo')

## Results

In [11]:
# Print set up again to compare
print('------------- Inventory -------------')
print('Budget:', Wbudget)
print('Number of elements in the universe: {:d}'.format(len(U)))
print('The inventory universe is', U)
print('The price of each item is', V)
print('Bound on item quantity is', upper_bound)
print('------------------------------------------')



------------- Inventory -------------
Budget: 100
Number of elements in the universe: 8
The inventory universe is [0, 1, 4, 9, 11, 12, 13, 15]
The price of each item is [1, 2, 10, 1, 10, 8, 6, 4]
Bound on item quantity is [2, 6, 5, 2, 5, 4, 3, 2]
------------------------------------------


In [12]:
print('------------- Suppliers -------------')
print('There are {:d} suppliers:'.format(len(S)))
print('----------------------------')
for i in range(len(S)):
    print('Supplier{:d}'.format(i), S[i])
    print('Costs: ', W[i])
    print('----------------------------')
print('------------------------------------------')



------------- Suppliers -------------
There are 5 suppliers:
----------------------------
Supplier0 {1, 9, 11, 12, 13}
Costs:  [2, 2, 8, 7, 5]
----------------------------
Supplier1 {9, 12, 13, 1}
Costs:  [1, 7, 5, 2]
----------------------------
Supplier2 {1, 11, 13}
Costs:  [2, 8, 5]
----------------------------
Supplier3 {1, 4, 9, 11, 15}
Costs:  [3, 8, 1, 10, 5]
----------------------------
Supplier4 {0, 12, 13, 15}
Costs:  [1, 8, 5, 5]
----------------------------
------------------------------------------


In [13]:
# -------------- Process the results ---------------
print('------------- Solution -------------')
feasible_sols = sampleset.filter(lambda row: row.is_feasible == True)
if not len(feasible_sols):
    print("\nNo feasible solution found.\n")
else:
    sol = feasible_sols.first.sample
    choosing_plan = [int(sol[(j,len(U)+1)]) for j in range(len(S))]
    purchase_plan = [int(sol[(j,i)])*choosing_plan[j] for i in range(len(U)) for j in range(len(S))]
    purchase_plan = np.array(purchase_plan).reshape( len(U), len(S) )

    df = pd.DataFrame( purchase_plan, index=pd.Index(U, name='Item'),
                       columns=pd.Index(list(range(len(S))), name='Supplier'))
    print('Choosing', choosing_plan)
    print(df)
    cost = quicksum(W[j][i]*purchase_plan[ U.index(list(S[j])[i]) ][j] for j in range(len(S)) for i in range(len(W[j])) )
    profit = quicksum( purchase_plan[i][j]*V[i] for i in range(len(U)) for j in range(len(S)) )
    print('Profit: ', profit)
    print('Cost: ', cost)
    print('Net Profit: ', profit-cost)

------------- Solution -------------
Choosing [1, 0, 0, 1, 1]
Supplier  0  1  2  3  4
Item                   
0         0  0  0  0  2
1         1  0  0  0  0
4         0  0  0  5  0
9         0  0  0  1  0
11        4  0  0  0  0
12        1  0  0  0  0
13        1  0  0  0  1
15        0  0  0  0  1
Profit:  119
Cost:  99
Net Profit:  20
